# Installs

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


# Imports

In [1]:
import pandas as pd
import numpy as np
import os
from configparser import ConfigParser
from sqlalchemy import create_engine, text

# Data Load and Preparation

In [2]:
# Set variable parameters
columns_target = ['zip', 'ugds']
columns_rename = ['zip', 'enrollment_of_undergraduates']
columns_datatype = 'float'
county_prefix = '0001'

# Database parameters
database_table_name = 'enrollment_of_undergraduates_0001'
database_columns = ['id', 'location_id', 'enrollment_of_undergraduates', 'year']
database_datatypes = 'SERIAL', 'TEXT', 'REAL', 'VARCHAR(4)'
database_constraints = ['PRIMARY KEY', 'NOT NULL', 'NOT NULL', 'NOT NULL']
foreign_key_table = 'location'
foreign_key_home = 'location_id'
foreign_key_away = 'location_id'

In [3]:
# Create a list of file paths
file_paths = [
    r"../data/college_scorecard/MERGED2010_11_PP.csv",
    r"../data/college_scorecard/MERGED2011_12_PP.csv",
    r"../data/college_scorecard/MERGED2012_13_PP.csv",
    r"../data/college_scorecard/MERGED2013_14_PP.csv",
    r"../data/college_scorecard/MERGED2014_15_PP.csv",
    r"../data/college_scorecard/MERGED2015_16_PP.csv",
    r"../data/college_scorecard/MERGED2016_17_PP.csv",
    r"../data/college_scorecard/MERGED2017_18_PP.csv",
    r"../data/college_scorecard/MERGED2018_19_PP.csv",
    r"../data/college_scorecard/MERGED2019_20_PP.csv",
    r"../data/college_scorecard/MERGED2020_21_PP.csv",
    r"../data/college_scorecard/MERGED2021_22_PP.csv"
]
df_zip_to_fips = pd.read_csv(r"../data/kaggle_zips_fips/ZIP-COUNTY-FIPS_2017-06.csv", dtype=str, low_memory=False)

# Create a list of dataframes by reading each file
list_df_initial = [pd.read_csv(file_path, dtype={'ZIP': str}, low_memory=False) for file_path in file_paths]
print("Dataframe Loaded")

Dataframe Loaded


In [4]:
# Set all header text to lowercase
for index in range(len(list_df_initial)):
    list_df_initial[index].columns = list_df_initial[index].columns.str.lower()
df_zip_to_fips.columns = df_zip_to_fips.columns.str.lower()
print("Headers Set to Lowercase")

Headers Set to Lowercase


# Data Inspection

In [5]:
# Show dataframe sizes
for df in list_df_initial:
    print(f"Index {index}: {df.shape}")

Index 11: (7470, 3232)
Index 11: (7746, 3232)
Index 11: (7862, 3232)
Index 11: (7869, 3232)
Index 11: (7766, 3232)
Index 11: (7666, 3232)
Index 11: (7238, 3232)
Index 11: (7112, 3232)
Index 11: (6807, 3232)
Index 11: (6694, 3232)
Index 11: (6681, 3232)
Index 11: (6543, 3232)


In [6]:
# Print column NaN count
for index in range(len(list_df_initial)):
    print(f"<Index {index}>")
    for col in columns_target:
        try:
            print(f"NaN {col}: {list_df_initial[index][col].isna().sum()}")
        except:
            print(f"----------{col} NOT FOUND----------")
    print("__________________________________________")

<Index 0>
NaN zip: 0
NaN ugds: 560
__________________________________________
<Index 1>
NaN zip: 0
NaN ugds: 639
__________________________________________
<Index 2>
NaN zip: 0
NaN ugds: 723
__________________________________________
<Index 3>
NaN zip: 0
NaN ugds: 728
__________________________________________
<Index 4>
NaN zip: 0
NaN ugds: 729
__________________________________________
<Index 5>
NaN zip: 0
NaN ugds: 751
__________________________________________
<Index 6>
NaN zip: 0
NaN ugds: 743
__________________________________________
<Index 7>
NaN zip: 0
NaN ugds: 748
__________________________________________
<Index 8>
NaN zip: 0
NaN ugds: 765
__________________________________________
<Index 9>
NaN zip: 0
NaN ugds: 792
__________________________________________
<Index 10>
NaN zip: 0
NaN ugds: 841
__________________________________________
<Index 11>
NaN zip: 0
NaN ugds: 774
__________________________________________


# Transform Bridge Dataset

In [7]:
# Creates a new dataframe with the selected columns
reduced_df_zip_to_fips = df_zip_to_fips[['zip', 'stcountyfp']]
print("Reduced Dataframe Created")

Reduced Dataframe Created


In [8]:
%%capture --no-stdout
# Remove all characters after the 5th position in the 'zip' column
reduced_df_zip_to_fips['zip'] = reduced_df_zip_to_fips['zip'].apply(lambda x: x[:5])
print("Characters after the 5th position removed")

Characters after the 5th position removed


In [9]:
%%capture --no-stdout
# Rename columns
reduced_df_zip_to_fips.rename(columns={'stcountyfp': 'location_id'}, inplace=True)    
print("Columns Renamed")

Columns Renamed


In [10]:
# Sort and reset index
reduced_df_zip_to_fips = reduced_df_zip_to_fips.sort_values(by=['zip'])
reduced_df_zip_to_fips.reset_index(drop=True, inplace=True)
print("Sorting and Index Reset Complete")

Sorting and Index Reset Complete


# Transform ScoreCard Data

In [11]:
# Create an empty list to store the reduced DataFrames
reduced_df_list = []

# Loop through each dataframe in list_df_initial and create a new list of dataframes with the selected columns
for index in range(len(list_df_initial)):
    reduced_df = list_df_initial[index][columns_target]
    reduced_df_list.append(reduced_df)
print("Reduced Dataframe Created")

Reduced Dataframe Created


In [12]:
%%capture --no-stdout
# Create a dictionary for renaming columns
rename_dict = dict(zip(columns_target, columns_rename))

# Rename the columns
for index in range(len(reduced_df_list)):
    # Rename columns in a dataframe using a dictionary
    reduced_df_list[index].rename(columns=rename_dict, inplace=True)    
print("Columns Renamed")

Columns Renamed


In [13]:
%%capture --no-stdout
# Drop rows where 'zip is NaN
for index in range(len(reduced_df_list)):
    reduced_df_list[index].dropna(subset=['zip'], inplace=True)
print("NaN location_id Dropped")

NaN location_id Dropped


In [14]:
%%capture --no-stdout
# Remove all characters after the 5th position in the 'zip' column
for index in range(len(reduced_df_list)):
    reduced_df_list[index]['zip'] = reduced_df_list[index]['zip'].apply(lambda x: x[:5])
print("Characters after the 5th position removed")

Characters after the 5th position removed


In [15]:
# Combine all values with the same 'zip' within the 'enrollment_of_undergraduates' column
for index in range(len(reduced_df_list)):
    reduced_df_list[index] = reduced_df_list[index].groupby('zip')['enrollment_of_undergraduates'].sum().reset_index()
print("Zip Codes Combined")

Zip Codes Combined


In [16]:
%%capture --no-stdout
# Iterate over each DataFrame in the list
for df in reduced_df_list:
    # Calculate the median of the 'enrollment_of_undergraduates' column (excluding NaN values)
    median_value = df[columns_rename[1]].median(skipna=True)
    
    # Fill NaN values in the 'enrollment_of_undergraduates' column with the mean value
    df[columns_rename[1]].fillna(median_value, inplace=True)

print(f"NaN values in the '{columns_rename[1]}' column filled with the median value.")

NaN values in the 'enrollment_of_undergraduates' column filled with the median value.


# Merge Dataframes

In [17]:
# Merge score_card dataset with fips_zips dataset
merged_df_list = []
for index in range(len(reduced_df_list)):
    merged_df = pd.merge(reduced_df_list[index], reduced_df_zip_to_fips, on='zip', how='inner')
    merged_df_list.append(merged_df)
print("Merged Dataframe Created")

print(merged_df_list[0].shape)
# Filter the DataFrame to rows where 'stcountyfp' equals the desired value
result_df = merged_df_list[0][merged_df_list[0]['location_id'] == '72005']
# Display the resulting DataFrame
print(result_df)

Merged Dataframe Created
(6200, 3)
     zip  enrollment_of_undergraduates location_id
0  00602                         298.0       72005
3  00603                        6342.0       72005
4  00604                        2842.0       72005
5  00605                         201.0       72005


In [18]:
# Combine all values with the same 'stcountyfp' within the 'enrollment_of_undergraduates' column
for index in range(len(reduced_df_list)):
    merged_df_list[index] = merged_df_list[index].groupby('location_id')['enrollment_of_undergraduates'].sum().reset_index()
print("StCountyFP Codes Combined")

print(merged_df_list[0].shape)
# Filter the DataFrame to rows where 'stcountyfp' equals the desired value
result_df = merged_df_list[0][merged_df_list[0]['location_id'] == '72005']
# Display the resulting DataFrame
print(result_df)

StCountyFP Codes Combined
(1930, 2)
     location_id  enrollment_of_undergraduates
1889       72005                        9683.0


In [19]:
%%capture --no-stdout
year_counter = 2010
# Add year to the list
if 'year' not in columns_rename:
    columns_rename.append('year')
    # Adds a year column to the dataframes
    for index in range(len(merged_df_list)):
        merged_df_list[index]['year'] = year_counter
        year_counter += 1
print("Years Added")

Years Added


# Combine All Years Into One Dataset

In [20]:
# Concatenate all DataFrames in the list
dfCombined = pd.concat(merged_df_list, axis=0, join='inner')
print("Dataframes Combined")

Dataframes Combined


In [21]:
%%capture --no-stdout
# Adds a country code pefix to the location_id
dfCombined['location_id'] = '0001' + dfCombined['location_id']

In [22]:
# Sort and reset index
dfCombined = dfCombined.sort_values(by=['location_id', 'year'])
dfCombined.reset_index(drop=True, inplace=True)
print("Sorting and Index Reset Complete")

Sorting and Index Reset Complete


In [23]:
# Set column datatypes
dfCombined['location_id'] = dfCombined['location_id'].astype(str)
dfCombined['year'] = dfCombined['year'].astype(str)
dfCombined[columns_rename[1]] = dfCombined[columns_rename[1]].astype(columns_datatype)
print("Column Types Set")

Column Types Set


# Save Dataframe as CSV

In [24]:
# Define the path where the CSV file will be saved
folder_path = '../test_folder'
file_path = os.path.join(folder_path, 'dataset_combined.csv')

# Check if the folder exists, if not, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the DataFrame as a CSV file in the specified location
dfCombined.to_csv(file_path, index=False)

# Print a success message
print(f"The DataFrame has been saved as a CSV file at {file_path}")

The DataFrame has been saved as a CSV file at ../test_folder\dataset_combined.csv


# Upload to Database

In [25]:
# Read database.ini
filename='../database.ini'
section='postgresql'
parser = ConfigParser()
parser.read(filename)
config = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        config[param[0]] = param[1]
else:
    raise Exception(f'Section {section} not found in the {filename} file')

# Create an SQLAlchemy engine
db = create_engine(f"postgresql+psycopg2://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

In [26]:
# Format the table config
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {database_table_name} ("""    
for index in range(len(database_columns)):        
    create_table_query = create_table_query + f"\n        {database_columns[index]} {database_datatypes[index]} {database_constraints[index]}"
    if index == len(database_columns) - 1:
        if len(foreign_key_table) > 0:
            create_table_query = create_table_query + f",\n        FOREIGN KEY ({foreign_key_home}) REFERENCES {foreign_key_table} ({foreign_key_away})\n    );"
        else:
            create_table_query = create_table_query + f"\n    );"
    else:
        create_table_query = create_table_query + ','

print("___________________________")
print(create_table_query)
print("___________________________")

# Create the table
try:
    # Execute the query
    with db.connect() as connection:
        connection.execute(text(create_table_query))
        connection.commit()
        connection.close()
    print("Table created successfully!")

except Exception as e:
    print(f"Error creating table: {e}")

___________________________

    CREATE TABLE IF NOT EXISTS enrollment_of_undergraduates_0001 (
        id SERIAL PRIMARY KEY,
        location_id TEXT NOT NULL,
        enrollment_of_undergraduates REAL NOT NULL,
        year VARCHAR(4) NOT NULL,
        FOREIGN KEY (location_id) REFERENCES location (location_id)
    );
___________________________
Table created successfully!


In [28]:
try:
    # Create the connection
    conn = db.connect()

    # Insert the DataFrame values into the table
    dfCombined.to_sql(database_table_name, con=conn, if_exists='append', index=False) 

    # Commits the changes and closes the connection
    conn.commit()
    conn.close()
    print("Database Upload Complete")
except Exception as e:
    print(f"Error: {e}")

Database Upload Complete


In [29]:
try:
    # Execute the query
    with db.connect() as connection:
        query = text(f"SELECT * FROM {database_table_name}")
        result = connection.execute(query)
        for row in result.fetchall():
            print(row)

except Exception as e:
    print(f"Error querying the database: {e}")

(1, '000101001', 1644.0, '2010')
(2, '000101001', 1722.0, '2011')
(3, '000101001', 1513.0, '2012')
(4, '000101001', 1474.0, '2013')
(5, '000101001', 1267.0, '2014')
(6, '000101001', 1113.0, '2015')
(7, '000101001', 951.0, '2016')
(8, '000101001', 1149.0, '2017')
(9, '000101001', 747.0, '2018')
(10, '000101001', 795.0, '2019')
(11, '000101001', 399.0, '2020')
(12, '000101001', 400.0, '2021')
(13, '000101003', 17987.0, '2010')
(14, '000101003', 18849.0, '2011')
(15, '000101003', 19107.0, '2012')
(16, '000101003', 19122.0, '2013')
(17, '000101003', 19766.0, '2014')
(18, '000101003', 19592.0, '2015')
(19, '000101003', 19816.0, '2016')
(20, '000101003', 19283.0, '2017')
(21, '000101003', 21027.0, '2018')
(22, '000101003', 19997.0, '2019')
(23, '000101003', 17974.0, '2020')
(24, '000101003', 16899.0, '2021')
(25, '000101007', 2493.0, '2010')
(26, '000101007', 2525.0, '2011')
(27, '000101007', 2588.0, '2012')
(28, '000101007', 2610.0, '2013')
(29, '000101007', 2644.0, '2014')
(30, '000101007'